<a href="https://colab.research.google.com/github/NDU-CSC413/cuda1/blob/master/matrix_mult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive matrix multiplication
CUDA implementation of matrix multiplication without using shared memory
$$
C_{ij}=\sum_kA_{ik}B_{kj}
$$

In [4]:
%%writefile example1.cu
#include <iostream>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <chrono>
using Duration = std::chrono::duration<double, std::milli>;

#define TIMEIT(dur,...)\
   {\
    auto start = std::chrono::high_resolution_clock::now();\
    __VA_ARGS__\
    auto end = std::chrono::high_resolution_clock::now();\
     dur = std::chrono::duration<double, std::milli>(end - start);\
}
/**
 *  mat_mult()->__global__ void
 * Matrix multiplication without using shared memory
 * @param da
 * @param db
 * @param dc
 * @param width
 * @return 
 */
__global__ void mat_mult(float* da, float* db, float* dc, int width) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float result = 0;
    for (int k = 0; k < width; ++k) 
    {
        result += da[row * width + k] * db[k * width + col];
    }
    dc[row * width + col] = result;
}

float  time_kernel(float* da, float* db, float* dc, int width,
                         dim3 blocks_per_grid,dim3 threads_per_block) {
    cudaEvent_t kernel_start, kernel_end;
    cudaEventCreate(&kernel_start);
    cudaEventCreate(&kernel_end);
    /* warmup call*/
    mat_mult <<<blocks_per_grid, threads_per_block >> > (da, db, dc, width);
    float time = 0;
    float total = 0;
    const int num_trials=500;
    for (int i = 0; i < num_trials; ++i) {
        cudaEventRecord(kernel_start);
        mat_mult << <blocks_per_grid, threads_per_block>> > (da, db, dc, width);
        cudaEventRecord(kernel_end);
        cudaEventSynchronize(kernel_end);
        cudaEventElapsedTime(&time, kernel_start, kernel_end);
        total += time;
    }
    /* average time in milliseconds */
    return total / num_trials;
}
int main() {
    const int matrix_w = 1024;
    const int msize = matrix_w * matrix_w;
    float* a, * b, * c;

    float* da, * db, * dc;
    a = (float*)malloc(msize * sizeof(float));
    b = (float*)malloc(msize * sizeof(float));
    c = (float*)malloc(msize * sizeof(float));
    for (int i = 0; i < msize; ++i) {
        a[i] = 1;
        b[i] = 1;
        c[i] = 0;
    }

    cudaMalloc(&da, msize * sizeof(float));
    cudaMalloc(&db, msize * sizeof(float));
    cudaMalloc(&dc, msize * sizeof(float));
    cudaMemcpy(da, a, msize * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(db, b, msize * sizeof(float), cudaMemcpyHostToDevice);


    /* total number of threads per block is 1024 which is the maximum */
    dim3 threads_per_block(32, 32);
    dim3 blocks_per_grid(matrix_w / threads_per_block.x, matrix_w / threads_per_block.y);
    float gpu_time = time_kernel(da, db, dc, matrix_w, blocks_per_grid, threads_per_block);
    cudaMemcpy(c, dc, msize * sizeof(float), cudaMemcpyDeviceToHost);

    for (int i = 0; i < msize; ++i) {
        if (c[i] != matrix_w) {
            std::cout << "ERROR\n"; break;
        }
        else c[i] = 0;
    }
    std::cout << "Multiplicatio of two " << matrix_w << "x" << matrix_w << " matrices\n";
    std::cout << "GPU time " << gpu_time << "  milliseconds\n";
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);
    Duration d;
    TIMEIT(d,
        for (int i = 0; i < matrix_w; ++i) {
            for (int j = 0; j < matrix_w; ++j)
                for (int k = 0; k < matrix_w; ++k)
                    c[i * matrix_w + j] += a[i * matrix_w + k] * b[matrix_w * k + j];
        }
    )
	for (int i = 0; i < msize; ++i) {
		if (c[i] != matrix_w) {
			std::cout << "ERROR\n"; break;
		}
		else c[i] = 0;
	}
    std::cout <<"CPU time "<< d.count() << " milliseconds \n";
    std::cout << "gain = " << d.count()/gpu_time << "\n";
    

    free(a);
    free(b);
    free(c);


}

Overwriting example1.cu


In [5]:
!nvcc -O2 example1.cu -o example1

In [6]:
!./example1

Multiplicatio of two 1024x1024 matrices
GPU time 3.21765  milliseconds
CPU time 3374.68 milliseconds 
gain = 1048.8


In [7]:
%%writefile example2.cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>
#include <algorithm>
#include <chrono>
/**
 * Matrix multiplication using shared memory.
 * The matrix is assumed to be square.
 */
using Duration = std::chrono::duration<double, std::milli>;

#define TIMEIT(dur,...)\
   {\
    auto start = std::chrono::high_resolution_clock::now();\
    __VA_ARGS__\
    auto end = std::chrono::high_resolution_clock::now();\
     dur = std::chrono::duration<double, std::milli>(end - start);\
}
#define BLOCK_SIZE 32
__global__ void mult(float* da, float* db, float* dc, int width) {

	int by= blockIdx.y;
	int bx = blockIdx.x;
	int ty = threadIdx.y;
	int tx = threadIdx.x;
	int row = by * BLOCK_SIZE + ty;
	int col = bx * BLOCK_SIZE + tx;
	__shared__ float sa[BLOCK_SIZE][BLOCK_SIZE];
	__shared__ float sb[BLOCK_SIZE][BLOCK_SIZE];
	float res = 0.0;
	int ntiles = width / BLOCK_SIZE;
	for (int b = 0; b < ntiles; ++b) {
		
		/* copy from memory to shared memory */
		sa[ty][tx] = da[row * width + b * BLOCK_SIZE + tx];
		sb[ty][tx] = db[(b * BLOCK_SIZE + ty) * width + col];
		
		__syncthreads();
		for (int k = 0; k < BLOCK_SIZE; ++k) {
			res += sa[ty][k] * sb[k][tx];
		}
		__syncthreads();
	}
	dc[row* width + col] = res;
}


int main() {
	cudaEvent_t kernel_start,kernel_end;
	cudaEventCreate(&kernel_start);
	cudaEventCreate(&kernel_end);


	float* a, * b, * c;
	float* da, * db, * dc;

	const int matrix_width = 1024;
	const int size = matrix_width * matrix_width;
	a = (float*)malloc(size * sizeof(float));
	b = (float*)malloc(size * sizeof(float));
	c = (float*)malloc(size * sizeof(float));
	for (int i = 0; i < size; ++i) {
		a[i] = 1;
		b[i] = 1;
	}
	cudaMalloc(&da, size * sizeof(float));
	cudaMalloc(&db, size * sizeof(float));
	cudaMalloc(&dc, size * sizeof(float));
	cudaMemcpy(da, a, size * sizeof(float), cudaMemcpyHostToDevice);
	cudaMemcpy(db, b, size * sizeof(float), cudaMemcpyHostToDevice);
	dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
	dim3 gridSize(matrix_width/ BLOCK_SIZE, matrix_width / BLOCK_SIZE);
	mult << <gridSize, blockSize >> > (da, db, dc, matrix_width);
	float time = 0;
	float gpu_time = 0;
	const int num_trials = 500;
	for (int i = 0; i < num_trials; ++i) {
		cudaEventRecord(kernel_start,0);
		mult << <gridSize, blockSize >> > (da, db, dc, matrix_width);
		cudaEventRecord(kernel_end,0);
		cudaEventSynchronize(kernel_end);
		cudaEventElapsedTime(&time, kernel_start, kernel_end);
		gpu_time += time;
	}
	gpu_time /= num_trials;
	std::cout << "GPU  time " << gpu_time << '\n';
	cudaMemcpy(c, dc, size * sizeof(float), cudaMemcpyDeviceToHost);
	for (int i = 0; i < size; i++) {
		if (c[i] != matrix_width) {
			std::cout << "error\n";
			break;
		}
		else c[i] = 0;
	}
	cudaFree(da);
	cudaFree(db);
	cudaFree(dc);
	Duration d;
	TIMEIT(d,
		for (int i = 0; i < matrix_width; ++i) {
			for (int j = 0; j < matrix_width; ++j)
				for (int k = 0; k < matrix_width; ++k)
					c[i * matrix_width + j] += a[i * matrix_width+ k] * b[matrix_width * k + j];
		}
	)
		
	std::cout << "CPU time " << d.count() << " milliseconds \n";
	std::cout << "gain = " << d.count() / gpu_time << "\n";
	free(a);
	free(b);
	free(c);

}

Writing example2.cu


In [8]:
!nvcc -O2 example2.cu -o example2
!./example2

GPU  time 2.32981
CPU time 3435.19 milliseconds 
gain = 1474.45


In [9]:
%%writefile cuda_device.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <iostream>
int getSPcores(cudaDeviceProp devProp)
{
    int cores = 0;
    int mp = devProp.multiProcessorCount;
    switch (devProp.major) {
    case 2: // Fermi
        if (devProp.minor == 1) cores = mp * 48;
        else cores = mp * 32;
        break;
    case 3: // Kepler
        cores = mp * 192;
        break;
    case 5: // Maxwell
        cores = mp * 128;
        break;
    case 6: // Pascal
        if ((devProp.minor == 1) || (devProp.minor == 2)) cores = mp * 128;
        else if (devProp.minor == 0) cores = mp * 64;
        else printf("Unknown device type\n");
        break;
    case 7: // Volta and Turing
        if ((devProp.minor == 0) || (devProp.minor == 5)) cores = mp * 64;
        else printf("Unknown device type\n");
        break;
    case 8: // Ampere
        if (devProp.minor == 0) cores = mp * 64;
        else if (devProp.minor == 6) cores = mp * 128;
        else printf("Unknown device type\n");
        break;
    default:
        printf("Unknown device type\n");
        break;
    }
    return cores;
}

int main()
{
	int device;

	cudaDeviceProp properties;
	cudaError_t err = cudaSuccess;
	err = cudaGetDevice(&device);
	err = cudaGetDeviceProperties(&properties, device);
	std::cout << "processor count " << properties.multiProcessorCount << std::endl;
	std::cout << "warp size " << properties.warpSize << std::endl;
	std::cout << "name= " << properties.name << std::endl;
	std::cout << "Compute capability " << properties.major << "." << properties.minor << "\n";
	std::cout << "shared Memory/SM " << properties.sharedMemPerMultiprocessor
		<< std::endl;
    std::cout << "number of cores " << getSPcores(properties)<<"\n";
	//  std::cout<<"max blocks/SM "<<properties.maxBlocksPerMultiProcessor
	 // <<std::endl;
	if (err == cudaSuccess)
		printf("device =%d\n", device);
	else
		printf("error getting deivce\n");
	return 0;
}


Writing cuda_device.cu


In [10]:
!nvcc cuda_device.cu -o cuda_device
!./cuda_device

processor count 40
warp size 32
name= Tesla T4
Compute capability 7.5
shared Memory/SM 65536
number of cores 2560
device =0


In [11]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin